# [자동차 가격 예측 - 기계학습 5주차](https://www.kaggle.com/t/644c4141565843d5a7d46873bffc2679)
> 17011520 조영래

In [65]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from itertools import product

RANDOM_STATE = 915

In [66]:
file_path = '/content/drive/MyDrive/2021-1/기계학습(최유경)/5/2021-ml-p6/'

train = pd.read_csv(file_path + 'train_data.csv')
test = pd.read_csv(file_path + 'test_data.csv')
sample_submit = pd.read_csv(file_path + 'sample_submit.csv')

## 데이터 확인

In [67]:
train.shape, test.shape

((34170, 11), (16831, 10))

In [68]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34170 entries, 0 to 34169
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            34170 non-null  int64  
 1   company       34170 non-null  int64  
 2   model         34170 non-null  object 
 3   year          34170 non-null  int64  
 4   transmission  34170 non-null  int64  
 5   mileage       34170 non-null  int64  
 6   fueltype      34170 non-null  int64  
 7   tax           34170 non-null  int64  
 8   mpg           34170 non-null  float64
 9   engineSize    34170 non-null  float64
 10  price         34170 non-null  int64  
dtypes: float64(2), int64(8), object(1)
memory usage: 2.9+ MB


In [69]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16831 entries, 0 to 16830
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            16831 non-null  int64  
 1   company       16831 non-null  int64  
 2   model         16831 non-null  object 
 3   year          16831 non-null  int64  
 4   transmission  16831 non-null  int64  
 5   mileage       16831 non-null  int64  
 6   fueltype      16831 non-null  int64  
 7   tax           16831 non-null  int64  
 8   mpg           16831 non-null  float64
 9   engineSize    16831 non-null  float64
dtypes: float64(2), int64(7), object(1)
memory usage: 1.3+ MB


In [70]:
train.head()

,ID,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize,price
0,0,4,Auris,2015,1,47541,4,145,46.3,1.6,8095
1,1,0,X2,2018,3,5000,0,145,50.4,2.0,21726
2,2,2,Focus,2017,3,47018,4,145,51.4,1.0,10490
3,3,2,Focus,2020,1,1550,4,145,49.6,1.0,17490
4,4,0,3 Series,2017,3,23505,0,145,64.2,2.0,18995


In [71]:
test.head()

,ID,company,model,year,transmission,mileage,fueltype,tax,mpg,engineSize
0,0,2,Ka+,2017,1,10150,4,145,57.7,1.2
1,1,2,Kuga,2013,1,50216,0,145,53.3,2.0
2,2,3,Q5,2016,1,34189,0,200,47.9,2.0
3,3,1,I10,2020,1,1900,4,145,56.5,1.0
4,4,0,3 Series,2014,0,79809,0,20,68.9,2.0


In [72]:
train.describe().drop(['ID', 'company'], axis=1)

,year,transmission,mileage,fueltype,tax,mpg,engineSize,price
count,34170.000000,34170.000000,34170.000000,34170.000000,34170.000000,34170.000000,34170.000000,34170.000000
mean,2016.973017,1.181738,23922.025929,2.283728,118.465320,56.378955,1.671378,16785.876529
std,2.154711,0.989000,21503.918830,1.923044,65.155117,18.546955,0.598782,9980.595784
min,1996.000000,0.000000,1.000000,0.000000,0.000000,1.100000,0.000000,495.000000
25%,2016.000000,1.000000,7948.250000,0.000000,125.000000,47.900000,1.200000,9999.000000
50%,2017.000000,1.000000,18362.000000,4.000000,145.000000,56.500000,1.600000,14490.000000
75%,2019.000000,1.000000,33297.750000,4.000000,145.000000,64.200000,2.000000,20490.000000
max,2060.000000,3.000000,190000.000000,4.000000,580.000000,470.800000,6.600000,145000.000000


In [73]:
X = train.drop(['ID', 'price'], axis=1)
y = train['price']

## 문자열 피처 확인, 정수 라벨링

In [74]:
X['model'].unique()

array([' Auris', ' X2', ' Focus', ' 3 Series', ' Fiesta', ' Aygo', ' Q3',
       ' Yaris', ' EcoSport', ' Kuga', ' A5', ' A1', ' X5', ' A4', ' A3',
       ' A6', ' I10', ' Tucson', ' B-MAX', ' X3', ' Santa Fe', ' Supra',
       ' I20', ' Prius', ' Kona', ' S-MAX', ' X1', ' 2 Series', ' Q2',
       ' Q5', ' 4 Series', ' C-HR', ' Ioniq', ' A7', ' 1 Series', ' RS6',
       ' IX20', ' i8', ' R8', ' Avensis', ' Ka+', ' 5 Series', ' X4',
       ' Corolla', ' Mondeo', ' X6', ' C-MAX', ' RAV4', ' GT86', ' TT',
       ' Grand Tourneo Connect', ' 7 Series', ' Galaxy', ' M3', ' RS5',
       ' Q7', ' Tourneo Custom', ' I40', ' I800', ' 8 Series', ' KA',
       ' 6 Series', ' RS3', ' X7', ' I30', ' Verso', ' PROACE VERSO',
       ' Grand C-MAX', ' Land Cruiser', ' A8', ' Z4', ' SQ5', ' M5',
       ' Urban Cruiser', ' Fusion', ' Edge', ' M4', ' Q8', ' IX35',
       ' RS4', ' Hilux', ' i3', ' Puma', ' Mustang', ' S4',
       ' Tourneo Connect', ' M2', ' Camry', ' S3', ' Z3', ' SQ7', ' M6',
       ' I

In [75]:
X['model'].value_counts()

 Fiesta           4393
 Focus            3074
 3 Series         1637
 Kuga             1491
 Yaris            1422
                  ... 
 S8                  3
 Urban Cruiser       3
 Verso-S             2
 Veloster            2
 S5                  2
Name: model, Length: 98, dtype: int64

## train 데이터에 없는 model 칼럼 값이 test 데이터에 있다.
> LabelEncoder fit 할 때 train, test 합친 것으로 해서 해결하자

```python
model_le = LabelEncoder()
model_encoded_train = model_le.fit_transform(X['model'])
model_encoded_test = model_le.transform(test['model'])
X['model'] = model_encoded_train
test['model'] = model_encoded_test
X['model']
```
```
---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py in _encode_python(values, uniques, encode)
     64         try:
---> 65             encoded = np.array([table[v] for v in values])
     66         except KeyError as e:

4 frames
KeyError: ' Ka+'

During handling of the above exception, another exception occurred:

ValueError                                Traceback (most recent call last)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py in _encode_python(values, uniques, encode)
     66         except KeyError as e:
     67             raise ValueError("y contains previously unseen labels: %s"
---> 68                              % str(e))
     69         return uniques, encoded
     70     else:

ValueError: y contains previously unseen labels: ' Ka+'
```

In [76]:
le = LabelEncoder()
le.fit(pd.concat([train['model'], test['model']], axis=0))

X['model'] = le.transform(X['model'])
test['model'] = le.transform(test['model'])

X['model']

0        15
1        87
2        26
3        26
4         2
         ..
34165     0
34166    25
34167    25
34168    33
34169    25
Name: model, Length: 34170, dtype: int64

In [77]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2,
                                                                random_state=RANDOM_STATE,)

In [78]:
print(X_train.shape)
print(X_validation.shape)
print(y_train.shape)
print(y_validation.shape)

(27336, 9)
(6834, 9)
(27336,)
(6834,)


## scaling 하지 않은 데이터

In [79]:
def search_params(model, params, metric,
                  X_train, X_test, y_train, y_test, verbose=True):
  print(f'<<<{model.__class__}>>>')
  
  metrics_with_params = []
  param_names = params.keys()

  for param in product(*params.values()):
    param_dict = dict(zip(param_names, param))
    model.set_params(**param_dict)
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_metric = metric(y_train, y_train_pred)
    test_metric = metric(y_test, y_test_pred)
    
    if verbose:
      print(f'##### {param_dict} #####')
      print(f'train {metric.__class__}: {train_metric}')
      print(f'test {metric.__class__}: {test_metric}')
      print('\n')

    metrics_with_params.append((train_metric, test_metric, param_dict))

  return metrics_with_params

In [80]:
params = {
    'weights': ['uniform', 'distance'],
    'p': [1, 2],
    'n_neighbors': [1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
}

In [81]:
mae_with_params = search_params(KNeighborsRegressor(), params, mean_absolute_error,
                                X_train, X_validation, y_train, y_validation,
                                verbose=False)

<<<<class 'sklearn.neighbors._regression.KNeighborsRegressor'>>>>


In [82]:
# train mae, validation mae, (weights, p, n_neighbors) 순서로 출력
# train mae 오름차순 정렬하여 10개 출력

for mae_params in sorted(mae_with_params,
                         key= lambda x: x[0])[:10]:
  print(mae_params)

(15.593098060119573, 4087.166291207086, {'weights': 'distance', 'p': 1, 'n_neighbors': 7})
(15.593098060119573, 4096.007958645538, {'weights': 'distance', 'p': 1, 'n_neighbors': 8})
(15.593098060119573, 4104.929583537073, {'weights': 'distance', 'p': 1, 'n_neighbors': 9})
(15.593098060119573, 4113.813328554018, {'weights': 'distance', 'p': 1, 'n_neighbors': 10})
(15.593098060119573, 4125.454884192548, {'weights': 'distance', 'p': 1, 'n_neighbors': 11})
(15.593098060119573, 4140.076287266191, {'weights': 'distance', 'p': 1, 'n_neighbors': 12})
(15.593098060119573, 4151.219806071759, {'weights': 'distance', 'p': 1, 'n_neighbors': 13})
(15.593098060119573, 4163.484962170043, {'weights': 'distance', 'p': 1, 'n_neighbors': 14})
(15.593098060119573, 4182.988114415597, {'weights': 'distance', 'p': 1, 'n_neighbors': 15})
(15.593098060119573, 4503.257905256895, {'weights': 'distance', 'p': 2, 'n_neighbors': 7})


In [83]:
# validataion mae 오름차순 정렬하여 10개 출력

for mae_params in sorted(mae_with_params, key= lambda x: x[1])[:10]:
  print(mae_params)

(15.593098060119573, 4087.166291207086, {'weights': 'distance', 'p': 1, 'n_neighbors': 7})
(15.593098060119573, 4096.007958645538, {'weights': 'distance', 'p': 1, 'n_neighbors': 8})
(15.63818773778168, 4100.569947388112, {'weights': 'distance', 'p': 1, 'n_neighbors': 6})
(15.660736757389524, 4103.880003845244, {'weights': 'distance', 'p': 1, 'n_neighbors': 5})
(15.593098060119573, 4104.929583537073, {'weights': 'distance', 'p': 1, 'n_neighbors': 9})
(15.593098060119573, 4113.813328554018, {'weights': 'distance', 'p': 1, 'n_neighbors': 10})
(15.593098060119573, 4125.454884192548, {'weights': 'distance', 'p': 1, 'n_neighbors': 11})
(15.593098060119573, 4140.076287266191, {'weights': 'distance', 'p': 1, 'n_neighbors': 12})
(15.593098060119573, 4151.219806071759, {'weights': 'distance', 'p': 1, 'n_neighbors': 13})
(15.596027216856893, 4156.382281780243, {'weights': 'distance', 'p': 1, 'n_neighbors': 3})


## 표준화된 데이터

In [84]:
std_scaler = StandardScaler()
std_scaler.fit(X_train)

X_train_std = std_scaler.transform(X_train)
X_validation_std = std_scaler.transform(X_validation)

In [85]:
mae_with_params_std = search_params(KNeighborsRegressor(), params, mean_absolute_error,
                                    X_train_std, X_validation_std, y_train, y_validation, verbose=False)

<<<<class 'sklearn.neighbors._regression.KNeighborsRegressor'>>>>


In [86]:
# validataion mae 오름차순 정렬하여 10개 출력

for mae_params in sorted(mae_with_params_std, key= lambda x: x[1])[:10]:
  print(mae_params)

(15.593098060119573, 1294.5709113543735, {'weights': 'distance', 'p': 1, 'n_neighbors': 8})
(15.593098060119573, 1298.0034630164969, {'weights': 'distance', 'p': 1, 'n_neighbors': 7})
(15.593901814457126, 1298.86346482134, {'weights': 'distance', 'p': 1, 'n_neighbors': 5})
(15.593490878938644, 1299.5793160460162, {'weights': 'distance', 'p': 1, 'n_neighbors': 6})
(15.593098060119573, 1300.8643212518466, {'weights': 'distance', 'p': 1, 'n_neighbors': 9})
(15.593098060119573, 1304.0241111653056, {'weights': 'distance', 'p': 1, 'n_neighbors': 10})
(15.593098060119573, 1306.5090539515795, {'weights': 'distance', 'p': 1, 'n_neighbors': 11})
(15.593098060119573, 1310.5400295548748, {'weights': 'distance', 'p': 1, 'n_neighbors': 12})
(15.593098060119573, 1314.9903353005843, {'weights': 'distance', 'p': 1, 'n_neighbors': 13})
(15.593098060119573, 1318.0551388206645, {'weights': 'distance', 'p': 1, 'n_neighbors': 14})


## 정규화된 데이터

In [87]:
minmax_scaler = MinMaxScaler()
minmax_scaler.fit(X_train)

X_train_minmax = minmax_scaler.transform(X_train)
X_validation_minmax = minmax_scaler.transform(X_validation)

In [88]:
mae_with_params_minmax = search_params(KNeighborsRegressor(), params, mean_absolute_error,
                                    X_train_minmax, X_validation_minmax, y_train, y_validation, verbose=False)

<<<<class 'sklearn.neighbors._regression.KNeighborsRegressor'>>>>


In [89]:
# validataion mae 오름차순 정렬하여 10개 출력

for mae_params in sorted(mae_with_params_minmax, key= lambda x: x[1])[:10]:
  print(mae_params)

(15.593748170910157, 1323.6538193972713, {'weights': 'distance', 'p': 1, 'n_neighbors': 5})
(15.63818773778168, 1329.011185379532, {'weights': 'distance', 'p': 1, 'n_neighbors': 6})
(15.593098060119573, 1333.008282339915, {'weights': 'distance', 'p': 1, 'n_neighbors': 7})
(15.593098060119573, 1337.000758667011, {'weights': 'distance', 'p': 1, 'n_neighbors': 8})
(15.593098060119573, 1344.2209437881957, {'weights': 'distance', 'p': 1, 'n_neighbors': 9})
(15.593098060119573, 1345.0859910048778, {'weights': 'distance', 'p': 1, 'n_neighbors': 10})
(15.724161057457806, 1350.3554005558344, {'weights': 'distance', 'p': 1, 'n_neighbors': 3})
(15.593098060119573, 1350.998776482747, {'weights': 'distance', 'p': 1, 'n_neighbors': 11})
(15.593098060119573, 1356.194910035766, {'weights': 'distance', 'p': 1, 'n_neighbors': 12})
(15.593098060119573, 1361.4239898311644, {'weights': 'distance', 'p': 1, 'n_neighbors': 13})


## 제출데이터 만들기

In [90]:
# submission 만드는 함수
# params : (n_parameters, )의 형태로 dict 형태로 parameter 들어있음

def submission_by_params(model, params, X, y, test, submission, file_name_prefix='submission'):
  for param in params:
    model.set_params(**param)
    model.fit(X, y)
    
    y_pred = model.predict(test)
    sample_submit.iloc[:, -1] = y_pred
    
    file_name = f'{file_name_prefix}_{param}.csv'
    sample_submit.to_csv(file_name, index=False)

In [91]:
X_test = test.drop('ID', axis=1)

In [92]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16831 entries, 0 to 16830
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   company       16831 non-null  int64  
 1   model         16831 non-null  int64  
 2   year          16831 non-null  int64  
 3   transmission  16831 non-null  int64  
 4   mileage       16831 non-null  int64  
 5   fueltype      16831 non-null  int64  
 6   tax           16831 non-null  int64  
 7   mpg           16831 non-null  float64
 8   engineSize    16831 non-null  float64
dtypes: float64(2), int64(7)
memory usage: 1.2 MB


### 표준화된 데이터

In [93]:
std_scaler_submission = StandardScaler()
std_scaler_submission.fit(X)

X_std = std_scaler_submission.transform(X)
X_test_std = std_scaler_submission.transform(X_test)

In [94]:
submission_params_std = [x[2] for x in sorted(mae_with_params_std, key= lambda x: x[1])[:10]]

In [95]:
submission_by_params(KNeighborsRegressor(),
                     submission_params_std,
                     X_std, y, X_test_std, sample_submit,
                     file_name_prefix='std_submission')

#### 결과

##### kaggle 제출 결과
|weights|p|n_neighbors|socre|
|-|-|-|-|
|distance|1|7|1276.66981|
|distance|1|5|1283.53995|
|distance|1|10|1281.10507|
|distance|1|3|1326.98520|
|distance|1|3|1276.63917|

보기용 하이퍼파라미터별 validation 셋 점수
```
(15.593098060119573, 1294.5709113543735, {'weights': 'distance', 'p': 1, 'n_neighbors': 8})
(15.593098060119573, 1298.0034630164969, {'weights': 'distance', 'p': 1, 'n_neighbors': 7})
(15.593901814457126, 1298.86346482134, {'weights': 'distance', 'p': 1, 'n_neighbors': 5})
(15.593490878938644, 1299.5793160460162, {'weights': 'distance', 'p': 1, 'n_neighbors': 6})
(15.593098060119573, 1300.8643212518466, {'weights': 'distance', 'p': 1, 'n_neighbors': 9})
(15.593098060119573, 1304.0241111653056, {'weights': 'distance', 'p': 1, 'n_neighbors': 10})
(15.593098060119573, 1306.5090539515795, {'weights': 'distance', 'p': 1, 'n_neighbors': 11})
(15.593098060119573, 1310.5400295548748, {'weights': 'distance', 'p': 1, 'n_neighbors': 12})
(15.593098060119573, 1314.9903353005843, {'weights': 'distance', 'p': 1, 'n_neighbors': 13})
(15.593098060119573, 1318.0551388206645, {'weights': 'distance', 'p': 1, 'n_neighbors': 14})
```

### 정규화된 데이터

In [96]:
minmax_scaler_submission = MinMaxScaler()
minmax_scaler_submission.fit(X)

X_minmax = minmax_scaler_submission.transform(X)
X_test_minmax = minmax_scaler_submission.transform(X_test)

In [97]:
submission_params_minmax = [x[2] for x in sorted(mae_with_params_minmax, key= lambda x: x[1])[:10]]

In [98]:
submission_by_params(KNeighborsRegressor(),
                     submission_params_minmax,
                     X_minmax, y, X_test_minmax, sample_submit,
                     file_name_prefix='minmax_submission')

#### 결과

##### kaggle 제출 결과
|weights|p|n_neighbors|socre|
|-|-|-|-|
|distance|1|5|1309.61075|
|distance|1|7|1307.73826|
|distance|1|10|1317.78386|

보기용 하이퍼파라미터별 validation 셋 점수
```
(15.593748170910157, 1323.6538193972713, {'weights': 'distance', 'p': 1, 'n_neighbors': 5})
(15.63818773778168, 1329.011185379532, {'weights': 'distance', 'p': 1, 'n_neighbors': 6})
(15.593098060119573, 1333.008282339915, {'weights': 'distance', 'p': 1, 'n_neighbors': 7})
(15.593098060119573, 1337.000758667011, {'weights': 'distance', 'p': 1, 'n_neighbors': 8})
(15.593098060119573, 1344.2209437881957, {'weights': 'distance', 'p': 1, 'n_neighbors': 9})
(15.593098060119573, 1345.0859910048778, {'weights': 'distance', 'p': 1, 'n_neighbors': 10})
(15.724161057457806, 1350.3554005558344, {'weights': 'distance', 'p': 1, 'n_neighbors': 3})
(15.593098060119573, 1350.998776482747, {'weights': 'distance', 'p': 1, 'n_neighbors': 11})
(15.593098060119573, 1356.194910035766, {'weights': 'distance', 'p': 1, 'n_neighbors': 12})
(15.593098060119573, 1361.4239898311644, {'weights': 'distance', 'p': 1, 'n_neighbors': 13})
```

## 타겟 값을 표준화 해보자

In [99]:
target_std_scaler = StandardScaler()
target_std_scaler.fit(y_train.values.reshape((-1, 1)))

y_train_std = target_std_scaler.transform(y_train.values.reshape(-1, 1))
y_validation_std = target_std_scaler.transform(y_validation.values.reshape(-1, 1))

y_train_std = y_train_std.reshape((-1, ))
y_validation_std = y_validation_std.reshape((-1, ))

In [100]:
y_train_std.shape

(27336,)

In [101]:
mae_with_params_y_std = search_params(KNeighborsRegressor(), params, mean_absolute_error,
                                    X_train_std, X_validation_std, y_train_std, y_validation_std, verbose=False)

<<<<class 'sklearn.neighbors._regression.KNeighborsRegressor'>>>>


In [102]:
# validataion mae 오름차순 정렬하여 10개 출력

for mae_params in sorted(mae_with_params_y_std, key= lambda x: x[1])[:10]:
  print(mae_params)

(0.0015474094018465894, 0.12846909522811686, {'weights': 'distance', 'p': 1, 'n_neighbors': 8})
(0.0015474094018465894, 0.12880973072555393, {'weights': 'distance', 'p': 1, 'n_neighbors': 7})
(0.0015474891638678318, 0.12889507456635377, {'weights': 'distance', 'p': 1, 'n_neighbors': 5})
(0.001547448383935428, 0.12896611336256453, {'weights': 'distance', 'p': 1, 'n_neighbors': 6})
(0.0015474094018465894, 0.1290936331876344, {'weights': 'distance', 'p': 1, 'n_neighbors': 9})
(0.0015474094018465894, 0.1294072006776287, {'weights': 'distance', 'p': 1, 'n_neighbors': 10})
(0.0015474094018465894, 0.12965379848748698, {'weights': 'distance', 'p': 1, 'n_neighbors': 11})
(0.0015474094018465894, 0.13005381967141752, {'weights': 'distance', 'p': 1, 'n_neighbors': 12})
(0.0015474094018465894, 0.13049545384350134, {'weights': 'distance', 'p': 1, 'n_neighbors': 13})
(0.0015474094018465894, 0.13079959518626075, {'weights': 'distance', 'p': 1, 'n_neighbors': 14})


In [103]:
submission_params_y_std = [x[2] for x in sorted(mae_with_params_y_std, key= lambda x: x[1])[:10]]

In [104]:
# submission 만드는 함수 -> 표준화된 타겟값으로 학습을 위해 조금 변경

def submission_by_params_y_std(model, params, X, y, test, submission, file_name_prefix='submission'):
  target_std_scaler = StandardScaler()
  y = target_std_scaler.fit_transform(y.values.reshape(-1, 1))
  y = y.reshape((-1, ))

  for param in params:
    model.set_params(**param)
    model.fit(X, y)
    
    y_pred = model.predict(test)
    y_pred = target_std_scaler.inverse_transform(y_pred.reshape((-1, 1))).reshape((-1, ))
    sample_submit.iloc[:, -1] = y_pred
    
    file_name = f'{file_name_prefix}_{param}.csv'
    sample_submit.to_csv(file_name, index=False)

In [105]:
submission_by_params_y_std(KNeighborsRegressor(),
                           submission_params_y_std,
                           X_std, y, X_test_std, sample_submit,
                           file_name_prefix='y_std_submission')

#### 결과
> 검증셋에 대해서는 약간의 성능이 증가된 것으로 보이긴 했다... 생각해보니 KNN 회귀 타겟값 표준화는 크게 도움이 되지 않을 것 같기도 하고...

##### kaggle 제출 결과
|weights|p|n_neighbors|socre|
|-|-|-|-|
|distance|1|8|1276.63917|
|distance|1|7|1276.66981|
|distance|1|6|1278.22146|
|distance|1|5|1283.53995|
|distance|1|9|1278.23437|
|distance|1|10|1281.10507|


보기용 하이퍼파라미터별 validation 셋 점수
```
(0.0015474094018465894, 0.12846909522811686, {'weights': 'distance', 'p': 1, 'n_neighbors': 8})
(0.0015474094018465894, 0.12880973072555393, {'weights': 'distance', 'p': 1, 'n_neighbors': 7})
(0.0015474891638678318, 0.12889507456635377, {'weights': 'distance', 'p': 1, 'n_neighbors': 5})
(0.001547448383935428, 0.12896611336256453, {'weights': 'distance', 'p': 1, 'n_neighbors': 6})
(0.0015474094018465894, 0.1290936331876344, {'weights': 'distance', 'p': 1, 'n_neighbors': 9})
(0.0015474094018465894, 0.1294072006776287, {'weights': 'distance', 'p': 1, 'n_neighbors': 10})
(0.0015474094018465894, 0.12965379848748698, {'weights': 'distance', 'p': 1, 'n_neighbors': 11})
(0.0015474094018465894, 0.13005381967141752, {'weights': 'distance', 'p': 1, 'n_neighbors': 12})
(0.0015474094018465894, 0.13049545384350134, {'weights': 'distance', 'p': 1, 'n_neighbors': 13})
(0.0015474094018465894, 0.13079959518626075, {'weights': 'distance', 'p': 1, 'n_neighbors': 14})
```